In [1]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.4 MB/s eta 0:00:00


In [6]:
import yfinance as yf

In [8]:
import pandas as pd

In [14]:
tickers = ["BTC-USD", "CL=F"]
start_date = "2024-01-01"
end_date = "2024-12-31"

data = yf.download(tickers, start=start_date, end=end_date)
display(data.head())

/tmp/ipython-input-1933864821.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  2 of 2 completed


Price              Close                     High                      Low  \
Ticker           BTC-USD       CL=F       BTC-USD       CL=F       BTC-USD   
Date                                                                         
2024-01-01  44167.332031        NaN  44175.437500        NaN  42214.976562   
2024-01-02  44957.968750  70.379997  45899.707031  73.639999  44176.949219   
2024-01-03  42848.175781  72.699997  45503.242188  73.230003  40813.535156   
2024-01-04  44179.921875  72.190002  44770.023438  74.000000  42675.175781   
2024-01-05  44162.691406  73.809998  44353.285156  74.239998  42784.718750   

Price                          Open                  Volume            
Ticker           CL=F       BTC-USD       CL=F      BTC-USD      CL=F  
Date                                                                   
2024-01-01        NaN  42280.234375        NaN  18426978443       NaN  
2024-01-02  70.059998  44187.140625  71.709999  39335274536  330992.0  
2024-01-03  69.279999  44961.601562  70.500000  46342323118  334861.0  
2024-01-04  71.059998  42855.816406  73.040001  30448091210  344467.0  
2024-01-05  72.209999  44192.980469  72.400002  32336029347  325525.0

In [15]:
# Select the 'Close' price and calculate daily returns
close_prices = data['Close']
daily_returns = close_prices.pct_change()*100

# Drop rows with NaN values
daily_returns = daily_returns.dropna()

display(daily_returns.head())

/tmp/ipython-input-3174793389.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = close_prices.pct_change()*100


Ticker,BTC-USD,CL=F
Date,,
2024-01-03,-4.692812,3.296391
2024-01-04,3.108058,-0.701506
2024-01-05,-0.039001,2.244071
2024-01-06,-0.392857,0.000000
2024-01-07,-0.104793,0.000000


In [23]:
import statsmodels.api as sm

# Define the dependent and independent variables
Y = daily_returns['BTC-USD']
X = daily_returns['CL=F']

# Add a constant (intercept) to the independent variable
X = sm.add_constant(X)

# Perform OLS regression
model = sm.OLS(Y, X). fit()

# Display the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                BTC-USD   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.057
Date:                Wed, 24 Sep 2025   Prob (F-statistic):              0.152
Time:                        04:18:14   Log-Likelihood:                -887.85
No. Observations:                 363   AIC:                             1780.
Df Residuals:                     361   BIC:                             1787.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2401      0.147      1.634      0.1

Predictive Model Using



# Task
Analyze the relationship between "CL=F" and "BTC-USD" using a Random Forest regression model. Download the data from Yahoo Finance for the period 2024-01-01 to 2024-12-31. Split the data into 80% training and 20% testing sets. Train the model to predict "BTC-USD" based on "CL=F". Evaluate the model's performance using RMSE and MAE.

## Prepare data

### Subtask:
Create the feature matrix (X) and the target vector (Y) from the `daily_returns` DataFrame, ensuring they are aligned and handle any remaining missing values if necessary.


**Reasoning**:
Create the feature matrix X and the target vector Y from the daily_returns DataFrame, aligning their indices and handling missing values.



In [24]:
# Create the feature matrix (X) and the target vector (Y)
X = daily_returns[['CL=F']]
Y = daily_returns['BTC-USD']

# Align indices and drop missing values
X, Y = X.align(Y, join='inner', axis=0)
X = X.dropna()
Y = Y.dropna()

display(X.head())
display(Y.head())

Ticker,CL=F
Date,
2024-01-03,3.296391
2024-01-04,-0.701506
2024-01-05,2.244071
2024-01-06,0.000000
2024-01-07,0.000000


,BTC-USD
Date,
2024-01-03,-4.692812
2024-01-04,3.108058
2024-01-05,-0.039001
2024-01-06,-0.392857
2024-01-07,-0.104793


## Split data

### Subtask:
Split the data into 80% for training and 20% for testing.


**Reasoning**:
Import the train_test_split function and split the data into training and testing sets.



In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (290, 1)
Shape of X_test: (73, 1)
Shape of Y_train: (290,)
Shape of Y_test: (73,)


## Train model

### Subtask:
Train a Random Forest Regressor model on the training data.


**Reasoning**:
Import the Random Forest Regressor and train the model.



In [26]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate a RandomForestRegressor object
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

## Predict

### Subtask:
Make predictions on the test data using the trained model.


**Reasoning**:
Use the trained Random Forest model to make predictions on the test set.



In [27]:
# Make predictions on the test set
Y_pred = rf_model.predict(X_test)

## Evaluate model

### Subtask:
Calculate and report the Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE) of the predictions.


**Reasoning**:
Calculate the RMSE and MAE of the predictions and print the results.



In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(Y_test, Y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(Y_test, Y_pred)

# Print the results
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Root Mean Squared Error (RMSE): 4.0358
Mean Absolute Error (MAE): 3.0455


## Summary:

### Data Analysis Key Findings

*   The daily returns of "CL=F" were used as the feature (`X`) and the daily returns of "BTC-USD" were used as the target (`Y`) for the analysis.
*   The data was split into an 80% training set and a 20% testing set.
*   A Random Forest Regressor model with 100 estimators was trained on the training data.
*   The model achieved a Root Mean Squared Error (RMSE) of 4.0358 and a Mean Absolute Error (MAE) of 3.0455 on the test set.

### Insights or Next Steps

*   The RMSE and MAE values indicate the average magnitude of the errors in the model's predictions of BTC-USD daily returns based on CL=F daily returns.
*   Further analysis could involve exploring other features, such as volume or volatility, or trying different regression models to potentially improve the prediction accuracy.
